# A Simple Example of a [Data-driven Journalism](https://en.wikipedia.org/wiki/Data-driven_journalism) Application
---

In [1]:
#Pkg.add("CSV")
#Pkg.add("DataFrames")
#Pkg.add("Query")
using CSV
using DataFrames
#using Query

# Question 1

_Which are the people that are affiliated to a Brazilian political party and also are public servants from a Brazilian federal university?_

## Data (acquisition)

* List of all public servants from the [Brazilian Official Transparency Web portal](http://transparencia.gov.br). 
* List of [all people affiliated to all Brazilian political parties](http://dados.gov.br/dataset/filiados-partidos-politicos) from the [Brazilian Offical Open Data Portal](http://dados.gov.br).

_Remark - BUGFIX: We faced problems when loading the origianl files so we created copied versions of these files to work on._

In [2]:
# Loading dataset: all people affiliate to the political party (ex.: PT)
path_ptal = "data-ddj/filiados-al/aplic-7/sead/lista_filiados/uf/filiados_pt_al-Unicode-200_lines.csv"
ptal = CSV.read(path_ptal,delim=";")

,DATA DA EXTRACAO,HORA DA EXTRACAO,NUMERO DA INSCRICAO,NOME DO FILIADO,SIGLA DO PARTIDO,NOME DO PARTIDO,UF,CODIGO DO MUNICIPIO,NOME DO MUNICIPIO,ZONA ELEITORAL,SECAO ELEITORAL,DATA DA FILIACAO,SITUACAO DO REGISTRO,TIPO DO REGISTRO,DATA DO PROCESSAMENTO,DATA DA DESFILIACAO,DATA DO CANCELAMENTO,DATA DA REGULARIZACAO,MOTIVO DO CANCELAMENTO
1,29/07/2018,00:57:37,24221531783,CARLOS HENRIQUE GUEDES DE ALMEIDA,PT,PARTIDO DOS TRABALHADORES,AL,27855,MACEIÓ,1,39,15/09/2004,REGULAR,OFICIAL,14/04/2018,missing,missing,missing,missing
2,29/07/2018,00:57:37,26591771740,DJALBA JOAQUIM DOS SANTOS,PT,PARTIDO DOS TRABALHADORES,AL,27197,BOCA DA MATA,48,56,25/09/2007,REGULAR,OFICIAL,15/04/2016,missing,missing,missing,missing
3,29/07/2018,00:57:37,17819191791,JOSE MATIAS DOS SANTOS,PT,PARTIDO DOS TRABALHADORES,AL,28932,TEOTÔNIO VILELA,34,132,03/07/2003,REGULAR,OFICIAL,14/04/2018,missing,missing,missing,missing
4,29/07/2018,00:57:37,4356561708,RICARDO JOSE MORONI VALENCA,PT,PARTIDO DOS TRABALHADORES,AL,27855,MACEIÓ,2,199,24/12/1987,CANCELADO,OFICIAL,14/04/2018,missing,02/03/2016,missing,Judicial
5,29/07/2018,00:57:37,2965641708,PEDRO CIRILO DOS SANTOS,PT,PARTIDO DOS TRABALHADORES,AL,28436,POÇO DAS TRINCHEIRAS,50,18,08/07/1999,REGULAR,OFICIAL,16/04/2015,missing,missing,missing,missing
6,29/07/2018,00:57:37,24600801767,REGIVANIO PINTO,PT,PARTIDO DOS TRABALHADORES,AL,28436,POÇO DAS TRINCHEIRAS,50,32,29/07/1999,REGULAR,OFICIAL,16/04/2015,missing,missing,missing,missing
7,29/07/2018,00:57:37,16963271708,MARINALVA DE OMENA SILVA,PT,PARTIDO DOS TRABALHADORES,AL,27855,MACEIÓ,2,93,02/04/2007,REGULAR,OFICIAL,15/04/2016,missing,missing,missing,missing
8,29/07/2018,00:57:37,25753411708,GEDIVAN COSTA,PT,PARTIDO DOS TRABALHADORES,AL,27855,MACEIÓ,3,56,02/09/2007,REGULAR,OFICIAL,15/04/2016,missing,missing,missing,missing
9,29/07/2018,00:57:37,18522211716,JOSILENE DA SILVA SANTOS,PT,PARTIDO DOS TRABALHADORES,AL,27855,MACEIÓ,3,37,07/07/1999,REGULAR,OFICIAL,15/04/2016,missing,missing,missing,missing
10,29/07/2018,00:57:37,31821091708,ASSIS ALEXANDRE DA SILVA,PT,PARTIDO DOS TRABALHADORES,AL,27278,CAMPO ALEGRE,47,55,10/05/2004,REGULAR,OFICIAL,16/10/2015,missing,missing,missing,missing


In [4]:
# Loading dataset: all public servants (ex.: only active public servants)
path_pservants = "data-ddj/servidores/201806_Servidores/20180630_Cadastro-350.csv"
pservants = CSV.read(path_pservants,delim=";")

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,DESCRICAO_CARGO,CLASSE_CARGO,REFERENCIA_CARGO,PADRAO_CARGO,NIVEL_CARGO,SIGLA_FUNCAO,NIVEL_FUNCAO,FUNCAO,CODIGO_ATIVIDADE,ATIVIDADE,OPCAO_PARCIAL,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_UORG_EXERCICIO,UORG_EXERCICIO,COD_ORG_EXERCICIO,ORG_EXERCICIO,COD_ORGSUP_EXERCICIO,ORGSUP_EXERCICIO,TIPO_VINCULO,SITUACAO_VINCULO,DATA_INICIO_AFASTAMENTO,DATA_TERMINO_AFASTAMENTO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,UF_EXERCICIO
1,5116961,AARAO CARLOS LUZ MACAMBIRA,***.017.623-**,016****,BIBLIOTECARIO-DOCUMENTALISTA,E,0,306,0,-1,-1,Sem informa袯,-1,Sem informa袯,missing,26405000000040,DIRETORIA GERAL/CAMPUS SOBRAL,26405,INSTITUTO FEDERAL DO CEARA,15000,MINISTERIO DA EDUCACAO,26405000000040,DIRETORIA GERAL/CAMPUS SOBRAL,26405,INSTITUTO FEDERAL DO CEARA,15000,MINISTERIO DA EDUCACAO,2,ATIVO PERMANENTE,missing,missing,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,11/02/2009,missing,29/12/2008,699,11/02/2009,missing,LEI,PORTARIA,CE
2,201964,AARAO DIAMANTINO OLIVEIRA,***.056.281-**,000****,ANALISTA DO BANCO CENTRAL,E,missing,IV,missing,-1,-1,Sem informa袯,-1,Sem informa袯,missing,25201000000000,BANCO CENTRAL DO BRASIL,25201,BANCO CENTRAL DO BRASIL,-1,Sem informa袯,25201000000000,BANCO CENTRAL DO BRASIL,25201,BANCO CENTRAL DO BRASIL,-1,Sem informa袯,2,ATIVO PERMANENTE,missing,missing,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,05/01/1998,missing,missing,000000000,05/01/1998,missing,Inv૩do,Inv૩do,DF
3,201964,AARAO DIAMANTINO OLIVEIRA,***.056.281-**,000****,Sem informa袯,missing,-1,-1,-1,FBC,FDT1,FUNCAO COMISSIONADA DO BANCO CENTRAL,FDT1,CHEFE DE SUBUNIDADE,missing,25201000000000,BANCO CENTRAL DO BRASIL,25201,BANCO CENTRAL DO BRASIL,-1,Sem informa袯,25201000000000,BANCO CENTRAL DO BRASIL,25201,BANCO CENTRAL DO BRASIL,-1,Sem informa袯,1,ATIVO PERMANENTE,missing,missing,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,04/06/2018,missing,missing,000000000,05/01/1998,missing,Inv૩do,Inv૩do,-1
4,4405000,AARAO FERREIRA LIMA NETO,***.116.132-**,014****,Sem informa袯,missing,-1,-1,-1,FG,0001,FUNCAO GRATIFICADA - IFES,0099,DIRETOR,missing,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,UNIVERSIDADE FEDERAL DO PARA,15000,MINISTERIO DA EDUCACAO,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,UNIVERSIDADE FEDERAL DO PARA,15000,MINISTERIO DA EDUCACAO,1,ATIVO PERMANENTE,missing,missing,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,13/04/2016,missing,24/07/2006,2475,16/08/2006,missing,PORTARIA,PORTARIA,-1
5,4405000,AARAO FERREIRA LIMA NETO,***.116.132-**,014****,PROFESSOR DO MAGISTERIO SUPERIOR,6,0,missing,603,-1,-1,Sem informa袯,-1,Sem informa袯,missing,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,UNIVERSIDADE FEDERAL DO PARA,15000,MINISTERIO DA EDUCACAO,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,UNIVERSIDADE FEDERAL DO PARA,15000,MINISTERIO DA EDUCACAO,2,ATIVO PERMANENTE,missing,missing,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,01/03/2013,missing,24/07/2006,2475,16/08/2006,missing,PORTARIA,PORTARIA,-1
6,4915841,AARAO PEREIRA DE ARAUJO JUNIOR,***.031.184-**,002****,PROFESSOR ENS BASICO TECN TECNOLOGICO,D,0,missing,501,-1,-1,Sem informa袯,-1,Sem informa袯,missing,26417000000013,DIR. DESENVOLVIMENTO ENSINO-JP,26417,INSTITUTO FEDERAL DA PARAIBA,15000,MINISTERIO DA EDUCACAO,26417000000240,"UNID. ACAD. I DES, ESTR.M.AMB-JP",26417,INSTITUTO FEDERAL DA PARAIBA,15000,MINISTERIO DA EDUCACAO,2,ATIVO PERMANENTE,missing,missing,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,01/03/2013,missing,29/12/2008,000000124,03/05/1993,missing,LEI,PORTARIA,PB
7,5105492,AARAO SOARES,***.859.807-**,014****,AGENTE DE COMBATE AS ENDEMIAS,S,0,V,0,-1,-1,Sem informa袯,-1,Sem informa袯,missing,25000000003648,SUS-SMS/DUQUE DE CAXIAS,25000,MINISTERIO DA SAUDE,-1,Sem i

## Filter 

* Getting all active ("regular" status) people from the party.
* Getting the names of all public servants from UFAL.

In [11]:
regulares_ptal = []
for row in eachrow(ptal)
    if row[Symbol("SITUACAO DO REGISTRO")] == "REGULAR"
        push!(regulares_ptal,row[Symbol("NOME DO FILIADO")])
    end
end

In [6]:
pservants_ufal = []
for row in eachrow(pservants)
    if row[Symbol("ORG_EXERCICIO")] == "UNIVERSIDADE FEDERAL DE ALAGOAS"
        #push!(pservants_ufal,row[Symbol("NOME")])
        if findfirst(regulares_ptal,row[Symbol("NOME")]) != 0
            push!(pservants_ufal,row[Symbol("NOME")])
        end
    end
end

# TODO Example using Query.jl
# query = @from i in pservants_ufal begin
#     @where i.Symbol("NOME") == 
# end

_Remark - DISCLAIMER: FAKE USER is mock data._

## Visualize

In [13]:
@show pservants_ufal

pservants_ufal = Any["FAKE USER"]


1-element Array{Any,1}:
 "FAKE USER"

## Story

FAKE USER was affiliated to TODO party in TODO and also is a public servant at TODO because she/he believes that TODO.

---
# Question 2
_How are distributed the political party affiliation in a Brazilian federal university?_

---
# Question 3
_How often people change political parties?_

---
# Question 4
_How often people join and/or quit a political party?_

---
# Question 5
_Which are the reasons for people quitting the parties? Which is the party that has more judicial reasons for quitting the party?_
